In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import os
import pandas as pd
import seaborn as sns
import fiona
import sklearn
import xgboost as xgb

In [3]:
#Upload the selection of glaccier i will use in the model
selection = []

with open('/Users/francesco/Desktop/Thesis/Data/dataset_filtered' + '/selecao.txt', 'r') as f:
    for line in f:
        selection.append(int(line.strip()))

common_years = np.arange(2005,2017,1)

print('The shape of the selection is: ', np.shape(selection))

The shape of the selection is:  (128,)


In [4]:
path_to_tif = '/Users/francesco/Desktop/Data/GEOTIFFs/'
shapefile_path = '/Users/francesco/Desktop//Thesis/Data/ice_shelf.shp'

ids = []
Names = []
regions = []
areas = []
lats = []
lons = []

shapefile=fiona.open(shapefile_path)
print(shapefile.schema['properties'])

for feature in shapefile:

    id = feature['properties']['id']
    name=feature['properties']['name']
    area=feature['properties']['area_fra']
    lat = feature['properties']['latitude']
    lon = feature['properties']['longitude']
    geometry=feature['geometry']
    if geometry is None:
        continue
    region = feature['properties']['regions']

    #Taking the info
    ids.append(id)
    Names.append(name)
    regions.append(region)
    areas.append(area)
    lats.append(lat)
    lons.append(lon)

#Creating the dataframe
df_region = pd.DataFrame(index=ids)
df_region ['id'] = ids
df_region ['Name'] = Names
df_region ['Region'] = regions
df_region ['Area'] = areas
df_region ['Latitude'] = lats
df_region ['Longitude'] = lons

#Selecting the region and sorting the dataframe
df_region = df_region[df_region['id'].isin(selection)]
df_region = df_region.sort_values(by=['id'])
#drop id column
df_region = df_region.drop(['id'], axis=1)

{'id': 'int:10', 'name': 'str:30', 'regions': 'str:11', 'type': 'str:2', 'is_index': 'int:10', 'ice_shelf_': 'str:50', 'measures_n': 'str:50', 'latitude': 'float:24.15', 'longitude': 'float:24.15', 'thickness_': 'float:24.15', 'thicknes_1': 'float:24.15', 'smb_mean_9': 'float:24.15', 'smb_mean_1': 'float:24.15', 'ss_gl_flux': 'float:24.15', 'ss_gl_fl_1': 'float:24.15', 'ss_calving': 'float:24.15', 'ss_calvi_1': 'float:24.15', 'model_gl_f': 'float:24.15', 'instant_co': 'float:24.15', 'mass_loss_': 'float:24.15', 'mass_los_1': 'float:24.15', 'mass_los_2': 'float:24.15', 'mass_los_3': 'float:24.15', 'mass_los_4': 'float:24.15', 'mass_los_5': 'float:24.15', 'mass_los_6': 'float:24.15', 'mass_los_7': 'float:24.15', 'mass_los_8': 'float:24.15', 'mass_los_9': 'float:24.15', 'mass_los10': 'float:24.15', 'mass_los11': 'float:24.15', 'mass_los12': 'float:24.15', 'mass_los13': 'float:24.15', 'mass_los14': 'float:24.15', 'mass_los15': 'float:24.15', 'mass_los16': 'float:24.15', 'mass_los17': 'floa

In [5]:
#Loading the dataset
dataset_directory = '/Users/francesco/Desktop/Thesis/Data/dataset_filtered/'

# Basal Melting
bm = pd.read_csv(dataset_directory + '/bm.csv', index_col=0)
bm = bm.sort_values(by=['id']) #sorting the glaciers by their index
bm = bm.loc[bm.index.isin(selection)] #selecting the glaciers, according to their index
bm = bm[common_years.astype(str)] #selecting the common years
bm = bm.sort_index() #sorting the glaciers by their index

#Load the calving data
calving = pd.read_csv(dataset_directory+ '/df_calving_from_shp_negative_and_positive.csv', index_col=0)
calving = calving.loc[calving.index.isin(selection)]
calving = calving[common_years.astype(str)]
#sort the calving by its index
calving = calving.sort_index()

#Load the ice concentration data
i_c = pd.read_csv(dataset_directory + '/ice_c_avg_extended_front.csv', index_col=0)
i_c = i_c.loc[i_c.index.isin(selection)]
i_c = i_c[common_years.astype(str)]
i_c = i_c.sort_index()

#Load the ice velocity data
i_v = pd.read_csv(dataset_directory + '/velocity_80_percentile_extended_front_2011_2012_linear_trend.csv', index_col=0)
i_v = i_v.loc[i_v.index.isin(selection)]
i_v = i_v[common_years.astype(str)]
i_v = i_v.sort_index()

#Load the ice thickness data
i_t = pd.read_csv(dataset_directory + '/thickness_avg_extended front.csv', index_col=0)
i_t = i_t.loc[i_t.index.isin(selection)]
i_t = i_t[common_years.astype(str)]
i_t = i_t.sort_index()

index = bm.index

In [6]:
#make a dataset with all the variables
dataset = pd.concat([bm, calving, i_c, i_v, i_t], axis=1, keys=['bm', 'calving', 'i_c', 'i_v', 'i_t'])
dataset.columns.names = ['Variables', 'Years']
dataset.index.names = ['Glaciers']

#i want to split the dataset in two, one with the variables and one with the target. The target is ice calving
dataset_target = dataset['calving']
dataset_variables = dataset.drop(['calving'], axis=1)

In [7]:
#Folders division

folder_1 = [2,4,6,7,8,15,18,16,34,45,135,163,95]
folder_2 = [3,29,10,12,31,30,19,24,35,50,86,104,139]
folder_3 = [65,43,37,17,41,32,38,40,36,52,143,146,125]
folder_4 = [69,67,63,26,46,33,64,44,70,53,123,145,170]
folder_5 = [75,78,89,39,47,81,77,54,71,55,118,101,117]
folder_6 = [76,80,114,58,48,84,83,66,90,56,140,144,164]
folder_7 = [88,82,120,61,57,92,85,105,100,73,158,124,169]
folder_8 = [96,107,121,62,60,127,87,108,102,91,161,122,160]

folder_9 = [129,112,156,68,97,132,93,109,115,98,148,166] #test
folder_10 = [131,147,157,72,99,136,116,111,119,110,162,159] #test

#merge the folders, in order to have a single list
from itertools import chain


train = [folder_1, folder_2, folder_3, folder_4, folder_5, folder_6, folder_7, folder_8]
test = [folder_9, folder_10]


In [8]:
#Operate block division
cv_block_1 = dataset.loc[folder_1]
cv_block_2 = dataset.loc[folder_2]
cv_block_3 = dataset.loc[folder_3]
cv_block_4 = dataset.loc[folder_4]
cv_block_5 = dataset.loc[folder_5]
cv_block_6 = dataset.loc[folder_6]
cv_block_7 = dataset.loc[folder_7]
cv_block_8 = dataset.loc[folder_8]

cv_blocks = [cv_block_1, cv_block_2, cv_block_3, cv_block_4, cv_block_5, cv_block_6, cv_block_7, cv_block_8]


In [20]:
#display all the columns options
pd.set_option('display.max_columns', None)
train_blocks[0]

Variables     bm                                                          \
Years       2005  2006   2007   2008   2009   2010   2011   2012    2013   
Glaciers                                                                   
3.0         0.03 -0.01   0.03   0.04   0.03  -0.02   0.01   0.02    0.03   
29.0        0.32  0.70  -0.18   0.38  -1.07  -0.05  -0.79   0.10    0.26   
10.0       11.84 -4.18   9.50   6.72   2.33  -2.37  12.24   6.46    4.06   
12.0        4.34  3.79   2.85   2.71   6.51   4.51   4.22   6.73    7.78   
31.0        3.95  4.59   1.03   5.77   0.73   2.21   3.54   3.44    2.64   
30.0        0.59 -0.30   0.37  -0.20  -0.27  -0.25   0.39   0.19   -0.11   
19.0       -0.12  0.13   0.32  -0.08   0.58   0.57   0.08   0.42   -0.07   
24.0        3.66  3.63   4.63   1.40   5.13   2.94   3.46   7.63    5.59   
35.0      -59.93  4.67  55.91  89.39 -34.18  37.19  20.23  62.67  121.96   
50.0       14.31  7.72  22.71  10.15  57.94  33.91  22.62  13.15    9.30   
86.0        0.90  3.72   0.06   5.44   5.64   8.94   4.07   2.77    6.18   
104.0       5.90  2.19   2.13 -15.95  21.76   4.35   5.96   1.97    1.21   
139.0       0.26 -0.32  -0.49   0.20   0.52   0.22   0.08   0.05    0.05   

Variables                       calving                                       \
Years        2014   2015   2016    2005   2006   2007    2008   2009    2010   
Glaciers                                                                       
3.0          0.04   0.01   0.02    0.12   0.14   0.13    0.01  -0.40    0.12   
29.0        -0.04  -0.06   0.47    0.49   0.29   0.38    0.29  -0.72   -2.21   
10.0         7.04   9.04   8.81    3.69   1.00   1.12    1.84   0.68    2.43   
12.0         7.48   8.81   5.65    3.50   0.86   0.27   -1.52  -3.01   -5.96   
31.0         2.86   2.40   2.20    0.33  -0.05  -0.05    0.08  -1.28    0.03   
30.0         0.23   0.29   0.22    0.01  -0.01   0.00    0.01   0.00    0.00   
19.0        -0.16   0.06   0.20    0.27   0.28   0.13    0.04  -0.72    0.28   
24.0         1.86   0.36  -0.40    0.03  -0.05   0.07   -0.01  -0.56    0.04   
35.0       100.36 -29.39  16.19   55.73  85.43  83.38  113.41  24.04   75.49   
50.0         8.85 -11.94  -5.21    0.74   5.84   4.23 -196.07 -15.86 -187.03   
86.0         5.53   7.88   6.11    0.86   0.31   0.59    1.21  -0.81    0.33   
104.0        4.89  -0.04   4.80    1.68  -2.33   1.80  -22.47  -1.82    3.59   
139.0        0.11   0.00   0.12    0.14   0.07   0.08    0.00  -0.56    0.12   

Variables                                                  i_c             \
Years       2011   2012   2013   2014   2015   2016       2005       2006   
Glaciers                                                                    
3.0        -0.30  -0.06   0.12   0.11  -0.43   0.00  63.666667  68.833333   
29.0        0.23   0.33   0.23   0.23  -1.32   0.12  88.226840  87.183002   
10.0        1.98   2.94   3.89   3.09  -0.33   4.31  91.554822  97.282716   
12.0        2.15   3.97   3.62   2.96   2.57   0.51  82.293391  76.185201   
31.0       -0.11  -0.05   0.06  -6.53  -0.05   0.14  85.044549  82.711530   
30.0        0.00   0.00   0.00   0.00   0.02   0.01  74.520325  81.412602   
19.0        0.25   0.13  -0.98   0.25  -0.04   0.56  91.355794  84.974284   
24.0        0.16  -0.61  -0.47  -0.83  -1.14   0.24  69.527365  67.641987   
35.0       93.30  78.95  88.91  60.23  95.82  84.39  91.226665  92.900400   
50.0       -1.58  -1.56  -4.51  -7.23 -16.61  -1.01  74.430460  81.335177   
86.0        0.47   0.51   0.61   0.56  -0.52   0.49  86.075633  84.078139   
104.0       4.36   4.44   3.94   3.70 -11.99  -5.52  76.797564  76.045166   
139.0       0.12   0.15   0.14  -0.50   0.12   0.12  85.750000  81.500000   

Variables                                                                    \
Years           2007       2008       2009       2010       2011       2012   
Glaciers                                                                      
3.0        64.833333  51.750000 

In [55]:
#Trying to immplement the cross validation in blocks

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

best_scores = []
best_parameters = []

rmse_results = []

grid = {'max_depth': [1, 2, 4, 6, 8],
          'n_estimators': [5, 10, 50, 100],
          'learning_rate': [0.001, 0.005,0.01]}

for i in range(len(cv_blocks)):
    print(f"Fold {i +1 }:")

    #Extracting the training an testing blocks from the list
    test_block = cv_blocks[i]
    train_blocks = cv_blocks[:i] + cv_blocks[i+1:]

    #Extracting the training and testing variables and targets
    X_test = test_block.drop(['calving'], axis=1)
    y_test = test_block['calving']

    X_train = pd.concat(train_blocks).drop(['calving'], axis=1)
    y_train = pd.concat(train_blocks)['calving']

    # Fit the model on the training data
    xgb_model_cv = GridSearchCV(xgb.XGBRegressor(),
                                grid,
                                n_jobs=-1,
                                scoring='neg_mean_squared_error'
                                )

    # Make predictions on the test data
    xgb_model_cv.fit(X_train, y_train)
    y_pred = xgb_model_cv.predict(X_test)

    # Evaluate the model's performance
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)


    best_scores.append(-xgb_model_cv.best_score_)
    print("Best score: %0.3f" % xgb_model_cv.best_score_)


    best_parameters.append(xgb_model_cv.best_params_)
    print("Best parameters: ", xgb_model_cv.best_params_)

Fold 1:
Best score: -1711.977
Best parameters:  {'learning_rate': 0.001, 'max_depth': 2, 'n_estimators': 5}
Fold 2:
Best score: -1881.253
Best parameters:  {'learning_rate': 0.001, 'max_depth': 8, 'n_estimators': 5}
Fold 3:
Best score: -2012.367
Best parameters:  {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 5}
Fold 4:
Best score: -1897.954
Best parameters:  {'learning_rate': 0.001, 'max_depth': 4, 'n_estimators': 50}
Fold 5:
Best score: -1783.822
Best parameters:  {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 10}
Fold 6:
Best score: -1028.175
Best parameters:  {'learning_rate': 0.005, 'max_depth': 6, 'n_estimators': 100}
Fold 7:
Best score: -1977.609
Best parameters:  {'learning_rate': 0.005, 'max_depth': 6, 'n_estimators': 50}
Fold 8:
Best score: -1636.846
Best parameters:  {'learning_rate': 0.005, 'max_depth': 2, 'n_estimators': 100}


In [56]:
rmse_results

[47.15060591503438,
 31.35616056652513,
 7.394023821800479,
 29.127562800234177,
 39.85531821408717,
 79.95982036011715,
 10.819692952689588,
 46.86037712089186]

In [50]:

grid = {'max_depth': [2, 4, 6, 8],
          'n_estimators': [10, 50, 100],#, 200, 300, 400], # number of trees (too many = overfitting, too few = underfitting)
          'learning_rate': [0.01,0.1,0,2]}#, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]}
''' 
Fold 1:
Best score: -0.087
Best parameters:  {'learning_rate': 0, 'max_depth': 2, 'n_estimators': 10}
Fold 2:
Best score: -0.090
Best parameters:  {'learning_rate': 0, 'max_depth': 2, 'n_estimators': 10}
Fold 3:
Best score: -0.090
Best parameters:  {'learning_rate': 0, 'max_depth': 2, 'n_estimators': 10}
Fold 4:
Best score: -0.091
Best parameters:  {'learning_rate': 0, 'max_depth': 2, 'n_estimators': 10}
Fold 5:
Best score: -0.079
Best parameters:  {'learning_rate': 0, 'max_depth': 2, 'n_estimators': 10}
Fold 6:
Best score: 0.045
Best parameters:  {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 50}
Fold 7:
Best score: -0.058
Best parameters:  {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 10}
Fold 8:
Best score: -0.040
Best parameters:  {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 10}
'''

#Second search

grid = {'max_depth': [1, 2, 4, 6, 8],
          'n_estimators': [5, 10, 50, 100, 200],
          'learning_rate': [0.001, 0.01,0.1]}
''' 
Fold 1:
Best score: -0.087
Best parameters:  {'learning_rate': 0.001, 'max_depth': 8, 'n_estimators': 5}
Fold 2:
Best score: -0.091
Best parameters:  {'learning_rate': 0.001, 'max_depth': 2, 'n_estimators': 5}
Fold 3:
Best score: -0.089
Best parameters:  {'learning_rate': 0.001, 'max_depth': 8, 'n_estimators': 10}
Fold 4:
Best score: -0.090
Best parameters:  {'learning_rate': 0.001, 'max_depth': 4, 'n_estimators': 10}
Fold 5:
Best score: -0.078
Best parameters:  {'learning_rate': 0.001, 'max_depth': 6, 'n_estimators': 10}
Fold 6:
Best score: 0.045
Best parameters:  {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 5}
Fold 7:
Best score: -0.058
Best parameters:  {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 10}
Fold 8:
Best score: -0.040
Best parameters:  {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 10}

rmse
[47.1365739506604,
 31.364554936950512,
 7.4516858846502805,
 29.373351773456424,
 39.47962844809016,
 80.03336934521863,
 10.723883194968833,
 46.05883447286754]
'''

type: invalid syntax (789630706.py, line 5)

In [49]:
rmse_results

[47.18678790938337,
 31.40842882542919,
 7.464980679041409,
 29.43826810201568,
 39.45603437647971,
 79.9642086810353,
 10.723883194968833,
 46.05883447286754]

In [17]:
#perform the cross validation with xgboost
import xgboost as xgb

#Create the parameters for the model
params = {
    'max_depth': 3,
    'learning_rate': 0.1,
    'objective': 'reg:squarederror',
    'n_estimators': 1000
}

#Create the model
model = xgb.XGBRegressor(**params)

#Create the cross validation

from sklearn.model_selection import cross_val_score

#Create the cross validation
scores = []
for i in range(len(cv_blocks)):
    score = cross_val_score(model, cv_block_variables[i], cv_block_targets[i], scoring='neg_mean_squared_error')
    scores.append(score)
    print('The score for the block ', i, ' is: ', score)



The score for the block  0  is:  [  -10.64023056 -2756.96996508 -6791.06483503 -4248.16741749
   -40.67776316]
The score for the block  1  is:  [-2097.97989868    -3.72819618 -2172.15697605 -3006.25288465
   -32.6781086 ]
The score for the block  2  is:  [-73.82074988  -5.85317016 -96.42872844 -55.05891223 -38.01497757]
The score for the block  3  is:  [-1.32432348e+01 -8.06455971e+01 -4.07830935e+03 -2.08674670e+02
 -2.03173624e+00]
The score for the block  4  is:  [-6.34946085e+00 -5.67810557e+00 -6.70933302e+03 -1.01090205e+04
 -2.05889142e+01]
The score for the block  5  is:  [   -67.40371266 -26726.20334862   -173.01548106 -40676.53896089
   -115.63300012]
The score for the block  6  is:  [ -30.13754018 -337.78433294 -350.12452017  -99.08480173  -57.6123823 ]
The score for the block  7  is:  [ -8674.12965997  -8487.02351694   -346.33354902 -12407.00786592
   -383.13696689]
